In [ ]:
# Dependencies
import pandas as pd
from pathlib import Path
import time
# used for interactive mapping 
import geopandas
import folium

>### <font color=darkblue>Import, Clean, & Narrow Station Data to Electric (ELEC) Stations only </font>
***
__Cleaned some of the data in CSV prior to import by removing unwanted columns__
  
- __Query Parameters:__1. _Passenger vehicles (class 1-2)_
2. _US only stations_
3. _Station is status 'Available'_
4. _Fuel Types (Biodiesel, Electric, Ethanol (E85), Renewable Diesel)_
5. _Station is 'public'_ic




In [ ]:
# Name of the Cleaned Station CSV file and read it in

US_stations_df = pd.read_csv('Resources/Cleaned_all_US_fuel_stations_1-28-RM.csv', encoding= 'ISO-8859-1')


In [ ]:
# review US_stations df
US_stations_df.head(2)

In [ ]:
# rename fuel type column, access_days_time, & station charging levels, cards_accepted 

renamed_US_stations_df = US_stations_df.rename(columns={"ï»¿Fuel _Type_Code": "Fuel_Type", "Access_Days_Time": "Business_Hours", "EV_Level1_EVSE_Num": "Count_Level_1","EV_Level2_EVSE_Num": "Count_Level_2", "EV_DC_Fast_Count": "Count_DC_Fast", "Cards_Accepted": "Payment"})
renamed_US_stations_df.head(2)

In [ ]:
# review data types in the df 
renamed_US_stations_df.dtypes

In [ ]:
# use value_counts() to count occurrences of 'Fuel_Types' (Biodiesel, Ele, Ethanol (E85), Renewable Diesel)

# Biodiesel ('BD')
count = renamed_US_stations_df['Fuel_Type'].value_counts()['BD']

print(f"The number stations for fuel type Biodiesel: {count}")

# Electic ('ELEC')
count = renamed_US_stations_df['Fuel_Type'].value_counts()['ELEC']

print(f"The number stations for fuel type Electric: {count}")

# Ethanol ('E85')
count = renamed_US_stations_df['Fuel_Type'].value_counts()['E85']

print(f"The number stations for fuel type Ethanol 85: {count}")


# Renewable Diesel ('RD')
count = renamed_US_stations_df['Fuel_Type'].value_counts()['RD']

print(f"The number stations for fuel type Renewable Diesel: {count}")


In [ ]:
#filter the df to only Electric stations 
ELEC_Stations_in_USA = renamed_US_stations_df.loc[(renamed_US_stations_df.Fuel_Type == "ELEC")]

#print ELEC Stations only DF
ELEC_Stations_in_USA.head()

In [ ]:
#count of rows in the elec stations df, to ensure I got all the electric stations 

len(ELEC_Stations_in_USA)

>### <font color=darkblue>RAGHEN- Create layer of makers for only DC fast stations on Choropleth of all Electric Stations in the US </font>
***
- Marker Hover Info: station name, address, phone, facility type, business hours
- should be able to turn markers on and off in the legend


In [ ]:
# refresher on the df info
ELEC_Stations_in_USA.head(2)

In [ ]:
# Make my df to add markers and hover- narrow df to only have stations with DC fast chargers 

#drop columns- payment, level 1, level 2,country, geo_code etc.)

new_DC_Fast_df= ELEC_Stations_in_USA.drop(['Payment', 'Count_Level_1', 'Count_Level_2', 'E85_Other_Ethanol_Blends', 'Country', 'Geocode_Status', 'EV_Connector_Types' ], axis=1)

new_DC_Fast_df.head()

In [ ]:
len(new_DC_Fast_df)

In [ ]:
# Ensure there are no null values for DC Fast- drop null values from DC Fast column

new_DC_Fast_df.dropna(subset=['Count_DC_Fast'], inplace=True)

In [ ]:
len(new_DC_Fast_df)

In [ ]:
# Reindex to update to more logical column order
new_DC_Fast_df = new_DC_Fast_df.reindex(columns=['Fuel_Type', 'Station_Name','Facility_Type', 'Street_Address',	'City',	'State', 'ZIP', 'Station_Phone', 'Business_Hours','Count_DC_Fast','Latitude', 'Longitude'])
new_DC_Fast_df.head(2)

In [ ]:
# Converting 'Facility_Type', 'Business_Hours', 'Station_Phone' to string
new_DC_Fast_df[['Facility_Type', 'Business_Hours', 'Station_Phone']] = new_DC_Fast_df[['Facility_Type', 'Business_Hours', 'Station_Phone']].astype(str)
# Displaying the types of data after conversion
print("\nTypes of data after conversion:\n", new_DC_Fast_df.dtypes)

In [ ]:
# use value_counts to count occurrences of 'DC-Fast Stations" per sate
new_DC_Fast_df['State'].value_counts()

In [ ]:
# Groupby: Group the DC-Fast Stations by State 
grouped_DC_Fast_df = new_DC_Fast_df.groupby(["State"], as_index= True) 


In [ ]:
# Calculate the sum of DC-Fast station ports per state 
Sum_of_DC_Fast_Stations = grouped_DC_Fast_df["Count_DC_Fast"].sum()

# Assemble into DataFrame
DC_Fast_summary_df = pd.DataFrame({
    "Count_DC_Fast":Sum_of_DC_Fast_Stations})


# Display results
DC_Fast_summary_df


In [ ]:
# find maximum and minimum value of station DC-Fast ports

#find the max ports by state
max_ports = DC_Fast_summary_df['Count_DC_Fast'].max() 
max_ports_index = DC_Fast_summary_df['Count_DC_Fast'].idxmax() 
print (f"The max port count is {max_ports} and occurs in the state of {max_ports_index}")

#find the min ports by state
min_ports= DC_Fast_summary_df['Count_DC_Fast'].min()
min_ports_index = DC_Fast_summary_df['Count_DC_Fast'].idxmin()
print (f"The min port count is {min_ports} and occurs in the state of {min_ports_index}")



In [ ]:
#create the base map using a central location around Montana, OpenStreetMap, and zoom of 3

base_map = folium.Map(location=[48, -102], tiles='OpenStreetMap',\
                      zoom_start=4)

#base_map

In [ ]:
# loop through the DataFrame to read the longitudinal and latitudinal values from the columns df[‘Longitude’] and df[‘Latitude’]

for i, row in new_DC_Fast_df.iterrows():
    lat = new_DC_Fast_df.at[i, 'Latitude']
    lng = new_DC_Fast_df.at[i, 'Longitude']


In [ ]:
# We will also create a pop-up to show the station name, facility type, business hours, address, phone within the loop.

#just station name to see if it works
popup = 'Station : ' + str(new_DC_Fast_df.at[i, 'Station_Name'])  


In [ ]:
# Withing the loop, pass the latitude and longitude values to the location variable, popup details, and folium icon details

#create map
base_map = folium.Map(location=[48, -102], tiles='OpenStreetMap', zoom_start=4)

#loop to get lat and lng values
for i, row in new_DC_Fast_df.iterrows():
    lat = new_DC_Fast_df.at[i, 'Latitude']
    lng = new_DC_Fast_df.at[i, 'Longitude']
    
    #hover state (station name, address, facility type, phone, hours)
    popup = 'Station : ' + str(new_DC_Fast_df.at[i, 'Station_Name']) + '<br>' + '<br>' +'Address: ' + new_DC_Fast_df.at[i, 'Street_Address'] + '<br>' + '<br>' +'Facility Type: ' + new_DC_Fast_df.at[i, 'Facility_Type']\
    + '<br>' + '<br>' +'Phone: ' + new_DC_Fast_df.at[i, 'Station_Phone'] + '<br>' + '<br>' +'Business Hours: ' + new_DC_Fast_df.at[i, 'Business_Hours']
    
    #add info to map and define the incon
    #https://fontawesome.com/icons?d=gallery for charging station icon
    #use with: prefix='fa'
    folium.Marker(location = [lat, lng], popup= popup, icon = folium.Icon(color='black', prefix='fa', icon='charging-station')).add_to(base_map)

#save as html file
base_map.save('DC_fast_Locations.html')
#base_map




>### <font color=darkblue>RAGHEN- Add US Average Winter Temps by State </font>
***
- add this data so that I can make Choropleth to then layer

In [ ]:
#import weather data file 
US_winter_temp_df = pd.read_csv('Resources/US_Average_Winter_Temps_F_c.csv')


In [ ]:
#take a look at the import 
US_winter_temp_df.head()


In [ ]:
#pip install geodatasets

In [ ]:
import geodatasets


In [ ]:
import geopandas as gpd
usmap_gdf = gpd.read_file('Resources/multipoly/cb_2018_us_state_500k.shp')

In [ ]:
usmap_gdf.head()

In [ ]:
# Rename US_winter_temp_df "State" column to Name so that so they can be merged 

#rename column "State" to "Name"
renamed_for_merge_US_winter_temp = US_winter_temp_df.rename(columns={"State": "NAME"})
renamed_for_merge_US_winter_temp.head()

In [ ]:
#do the merge to pull together temps and the geometry in one df

merged_FINAL = usmap_gdf.merge(renamed_for_merge_US_winter_temp, left_on='NAME',right_on='NAME')
merged_FINAL.head()


In [ ]:
#check to see if number of rows is correct post merge 
len(merged_FINAL)

In [ ]:
#view merged DF
#merged_FINAL.plot()

In [ ]:
#Creating maps for interactive exploration mirrors the API of static plots in an explore() method 
#of a GeoSeries or GeoDataFrame.
#merged_FINAL.explore()


In [ ]:
#add the data for the interactive choropleth
merged_FINAL.explore(
    # make choropleth based on avg temp values (F)
    column="Avg ° F",  
    
    # use mapclassify's natural breaks scheme
    scheme="naturalbreaks",
    
    # show legend
    legend=True,  
    
    # use 8 bins
    k=8, 
    
    # hide tooltip bc it will be layered and we added a pop up below that will make the user click to see temp value
    tooltip=False, 
    
    #pop up on click to show state temp 
    popup=["NAME", "Avg ° F", "Avg ° C"], 
    
     # do not use colorbar
    legend_kwds=dict(colorbar=False), 
    
    # name of the layer in the map
    name="US_Avg_Winter_Temps",  
 
)

In [ ]:
#layer the marker on the Choropleth 

m= merged_FINAL.explore(
  
    # make choropleth based on avg temp values (F)
    column="Avg ° F",  
    
    # use mapclassify's natural breaks scheme
    scheme="naturalbreaks",
    
    # show legend
    legend=True,  
    
    # use 8 bins
    k=8, 
    
    # hide tooltip bc it will be layered and we added a pop up below that will make the user click to see temp value
    tooltip=False, 
    
    #pop up on click to show state temp 
    popup=["NAME", "Avg ° F", "Avg ° C"], 
    
     # do not use colorbar
    legend_kwds=dict(colorbar=False), 
    
    # name of the layer in the map
    name="US States Avg Winter Temps",  
 
)

#markers added on the the choropleth map 
fg= folium.FeatureGroup(name= "Stations",overlay= True).add_to(m)
#loop to get lat and lng values
for i, row in new_DC_Fast_df.iterrows():
    lat = new_DC_Fast_df.at[i, 'Latitude']
    lng = new_DC_Fast_df.at[i, 'Longitude']
    
    #hover state (station name, address, facility type, phone, hours)
    popup = 'Station : ' + str(new_DC_Fast_df.at[i, 'Station_Name']) + '<br>' + '<br>' +'Address: ' + new_DC_Fast_df.at[i, 'Street_Address'] + '<br>' + '<br>' +'Facility Type: ' + new_DC_Fast_df.at[i, 'Facility_Type']\
    + '<br>' + '<br>' +'Phone: ' + new_DC_Fast_df.at[i, 'Station_Phone'] + '<br>' + '<br>' +'Business Hours: ' + new_DC_Fast_df.at[i, 'Business_Hours']

   
    #add info to map and define the incon
    #https://fontawesome.com/icons?d=gallery for charging station icon
    #use with: prefix='fa'
    folium.Marker(location = [lat, lng], popup= popup, icon = folium.Icon(color='black', prefix='fa', icon='charging-station')).add_to(fg)


folium.TileLayer('CartoDB positron', show=True).add_to(
    m
)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control

m  # show map



In [ ]:
#save as html file
m.save('Final_Layered_Map.html')